In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir /content/data
!unzip "/content/drive/MyDrive/Temp/Abhinaya/11_Oct-17_Oct/1550K MALWARE ANALYSIS DATASETS_API IMPORT.zip" -d /content/data

#!ls -l /content/data/ 
!head -100 "/content/data/MALWARE ANALYSIS DATASETS_API IMPORT.csv" > /content/sample.csv

# split file i train and test
#!wc -l "/content/data/MALWARE ANALYSIS DATASETS_API IMPORT.csv"  # 0.2*(1550001-310000)

!head -1240001 "/content/data/MALWARE ANALYSIS DATASETS_API IMPORT.csv" > /content/data/train.csv

!head -1 "/content/data/MALWARE ANALYSIS DATASETS_API IMPORT.csv" > /content/data/test.csv
!tail -150000 "/content/data/MALWARE ANALYSIS DATASETS_API IMPORT.csv" >> /content/data/test.csv


In [ ]:
# Check a sample set
import pandas as pd
sample_path = "/content/sample.csv"
data = pd.read_csv(sample_path)
#data.shape

# Create TF dataset
train_path="/content/data/train.csv"
test_path="/content/data/test.csv"

import tensorflow as tf
train = tf.data.experimental.make_csv_dataset(train_path, column_defaults=[tf.float32]*1001, select_columns=list(range(2,1003)), 
                                              label_name='label', batch_size=128)
test = tf.data.experimental.make_csv_dataset(test_path, column_defaults=[tf.float32]*1001, select_columns=list(range(2,1003)), 
                                              label_name='label', batch_size=128)

# for data,label in dataset:
#     #print(pd.DataFrame(data))
#     print(pd.Series(label).to_numpy())
#     break

def train_data():
    while True:
        for data,label in train:
            yield pd.DataFrame(data).to_numpy(), pd.Series(label).to_numpy()

def test_data():
    while True:
        for data,label in test:
            yield pd.DataFrame(data).to_numpy(), pd.Series(label).to_numpy()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [ ]:
n_timesteps=1000
n_features=1
n_outputs=1

model = Sequential()
model.add(tf.keras.layers.Reshape((1000, 1), input_shape=(1000,)))
model.add(Conv1D(filters=256, kernel_size=50, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=64, kernel_size=50, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_data(), steps_per_epoch=int(1240000/128),epochs=2)

9687/9687 [==============================] - 2142s 218ms/step - loss: 0.2302 - accuracy: 0.9008


In [38]:
import numpy as np, pandas as pd
def get_acc(test_path, model):
    test_data = pd.read_csv(test_path)
    y_test=test_data.label.to_numpy()
    test_data=test_data.iloc[:,3:]

    y_pred = model.predict(test_data)
    y_pred =  np.where(y_pred<0.5,0.,1.)

#    accuracy, precision, recall, F-1, confusion matrix
    from sklearn.metrics import precision_score,f1_score, recall_score, confusion_matrix, accuracy_score
    acc = accuracy_score(y_test,y_pred)#(y_pred.ravel()==y_test).sum()/y_test.shape[0]

    print("Accuracy - ",acc)
    print("Recall - ",recall_score(y_test,y_pred))
    print("Precision - ",precision_score(y_test,y_pred))
    print("F1 - ",f1_score(y_test,y_pred))
    print("CM - ",confusion_matrix(y_test,y_pred))

    return acc, y_pred

get_acc(test_path, model)

#del y_pred ;import gc ;gc.collect()


Accuracy -  0.82232
Recall -  0.7526084056037359
Precision -  0.8742986453392851
F1 -  0.808902400550664
CM -  [[66940  8110]
 [18542 56408]]


**Post-Training:**
<br>Once your model is trained, save and store the model. Then, create a function (or method) that takes a PE file as its argument, runs it through the trained model, and returns the output (i.e., Malware or Benign).

In [ ]:
# Save model
model_path="/content/drive/MyDrive/Temp/Abhinaya/11_Oct-17_Oct/model.h5"
#model.save(model_path)

from keras.models import load_model
def validate(path):
    model = load_model(model_path)

    acc, y_pred=get_acc(path, model)
    return y_pred

# Test for the sample data
y_pred_sample=validate("/content/sample.csv").ravel()

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]])

**Task 2 - Deploy your model on the cloud:**
<br>In this task, you will be using Amazon Sagemaker to deploy your model on the cloud, and create an endpoint (~ API) so that other applications can make use of the model.

In [ ]:
# Sagemaker is throwing error with this version of TF/Py

**Task 3 –Create a client:**
<br>This task is quite simple as well: create a Python code that *loads* a PE file, converts it into a feature vector that is compatiblewith your MalConv/EMBER model, runs the vector on the cloud API, and then prints the results (i.e., Malware or Benign –or probabilities of each).

In [ ]:
# We can write a sample client
import requests

url = 'http://aws.sagemaker.com/validate'

files = {'file': open('/content/sample.csv', 'rb')}

r = requests.post(url, files=files)

# Parse the response
r.text